In [39]:
%reload_ext autoreload
%autoreload 2

In [40]:
import os, sys
import re
import json
import glob
import datetime
from collections import Counter

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from wordcloud import WordCloud

In [41]:
# Add parent directory to path to import modules from src

rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from src.loader import SlackDataLoader
from src.loader import slack_parser
import src.utils as utils

In [42]:

data = SlackDataLoader("../anonymized")


In [38]:
# messages in a the document
messages = slack_parser(data.path +"/all-week1")
messages_df = pd.DataFrame(messages)
#messages_df.head(20)

../anonymized/all-week1
U03TEPYRM2P is the first element in the json


,msg_type,msg_content,sender_name,msg_sent_time,msg_dist_type,time_thread_start,reply_count,reply_users_count,reply_users,tm_thread_end,channel
0,message,<https://towardsdatascience.com/how-to-fill-mi...,Anthony Galloway,1661158502.784099,link,0,0,0,0,0,all-week1
1,message,"Imputation Using k-NN, Imputation Using Multiv...",Katie Dickerson,1661158692.172369,text,1661158692.172369,2,2,"U03U1GHT39V,U03UJGRN5E0",1661159006.632649,all-week1
2,message,<https://towardsdatascience.com/6-different-wa...,Katie Dickerson,1661158702.777369,link,0,0,0,0,0,all-week1
3,message,u can use SMOTE or mode to impute categorical ...,Joseph Diaz,1661158767.626859,text,1661158692.172369,0,0,0,0,all-week1
4,message,:+1:thanks,Katie Dickerson,1661159006.632649,emoji,1661158692.172369,0,0,0,0,all-week1
5,message,<https://www.scribbr.com/statistics/normal-dis...,Amy Leon,1661160070.855719,link,1661160070.855719,1,1,U03UFV7TUTV,1661161916.102399,all-week1
6,message,"Any have information, on the time the next ses...",Nancy Craig,1661161636.759979,text,1661161636.759979,5,4,"U03U93GNNVB,U03U1FNPEUX,U03UG5VFN03,U03UG0YHAUT",1661170044.469879,all-week1
7,message,Please check the schedule,Katherine Foster,1661161669.054589,text,1661161636.759979,0,0,0,0,all-week1
8,message,<https://www.notion.so/10academy/10-Academy-Ba...,Joshua Rhodes,1661161725.970779,link,1661161636.759979,0,0,0,0,all-week1
9,message,Thanks Natnael,Anthony Galloway,1661161916.102399,text,1661160070.855719,0,0,0,0,all-week1


In [48]:
#reply counts

reply_counts = messages_df.sort_values(by=['reply_count'],ascending=False)
reply_counts.head()

,msg_type,msg_content,sender_name,msg_sent_time,msg_dist_type,time_thread_start,reply_count,reply_users_count,reply_users,tm_thread_end,channel
394,message,I need some one who understood task two to ask...,Cristian Wilson,1661451551.074739,text,1661451551.074739,18,2,"U03U1FNPEUX,U03V6HMRPGQ",1661494027.744619,all-week1
68,message,<@U03UKL27B0R> I have used sum but it didn't s...,Veronica Padilla,1661206897.792509,user,1661206897.792509,14,4,"U03UJGP0C68,U03UUMM7Y8H,U03UG1Z21JP,U03V6HMRPGQ",1661363877.823639,all-week1
401,message,What is everybody using to measure session fre...,Steven Garcia,1661452626.763289,text,1661452626.763289,12,3,"U03UJGRN5E0,U03V6HMRPGQ,U03U1FNPEUX",1661454555.532449,all-week1
247,message,"What are you guys using for your work please, ...",Michelle Lewis,1661318940.506229,text,1661318940.506229,12,5,"U03UFV7TUTV,U03UG32J3PC,U03UJN29Y4C,U03UG1Z21J...",1661324676.252859,all-week1
372,message,I suggest that if the managing team made a sh...,Cristian Wilson,1661426750.680949,text,1661426750.680949,10,5,"U03V61VGQG0,U03UG4Q7V42,U03UH397319,U03UVHCV6K...",1661436050.217269,all-week1


In [27]:
#get users

user = data.get_users()
df = pd.DataFrame(user)
print(type(user))


<class 'list'>


In [28]:
# a channel
channels = data.get_channels()
df = pd.DataFrame(channels)
#df.head(20)


### Columns we can get from a slack message<br>

message_type, message_content, sender_id, time_sent, message_distribution, time_thread_start, reply_count, reply_user_count, time_thread_end, reply_users

From a single slack message, we can get <br>

1. The message<br>
2. Type (message, file, link, etc)<br>
3. The sender_id (assigned by slack)<br>
4. The time the message was sent<br>
5. The team (i don't know what that is now)<br>
6. The type of the message (broadcast message, inhouse, just messgae)<br>
7. The thread the message generated (from here we can go):<br>
    7.1 Text/content of the message<br>
    7.2 The thread time of the message<br>
    7.3 The thread count (reply count)<br>
    7.4 The number of user that reply the message (count of users that participated in the thread)<br>
    7.5 The time the last thread message was sent <br>
    7.6 The users that participated in the thread (their ids are stored as well)<br>

In [29]:
def get_top_20_user(data, channel='Random'):
    """get user with the highest number of message sent to any channel"""

    data['sender_name'].value_counts()[:20].plot.bar(figsize=(15, 7.5))
    plt.title(f'Top 20 Message Senders in #{channel} channels', size=15, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=14);
    plt.xticks(size=12); plt.yticks(size=12);
    plt.show()

    data['sender_name'].value_counts()[-10:].plot.bar(figsize=(15, 7.5))
    plt.title(f'Bottom 10 Message Senders in #{channel} channels', size=15, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=14);
    plt.xticks(size=12); plt.yticks(size=12);
    plt.show()

def draw_avg_reply_count(data, channel='Random'):
    """who commands many reply?"""

    data.groupby('sender_name')['reply_count'].mean().sort_values(ascending=False)[:20]\
        .plot(kind='bar', figsize=(15,7.5));
    plt.title(f'Average Number of reply count per Sender in #{channel}', size=20, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

def draw_avg_reply_users_count(data, channel='Random'):
    """who commands many user reply?"""

    data.groupby('sender_name')['reply_users_count'].mean().sort_values(ascending=False)[:20].plot(kind='bar',
     figsize=(15,7.5));
    plt.title(f'Average Number of reply user count per Sender in #{channel}', size=20, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

def draw_wordcloud(msg_content, week):    
    # word cloud visualization
    allWords = ' '.join([twts for twts in msg_content])
    wordCloud = WordCloud(background_color='#975429', width=500, height=300, random_state=21, max_words=500, mode='RGBA',
                            max_font_size=140, stopwords=stopwords.words('english')).generate(allWords)
    plt.figure(figsize=(15, 7.5))
    plt.imshow(wordCloud, interpolation="bilinear")
    plt.axis('off')
    plt.tight_layout()
    plt.title(f'WordCloud for {week}', size=30)
    plt.show()

def draw_user_reaction(data, channel='General'):
    data.groupby('sender_name')[['reply_count', 'reply_users_count']].sum()\
        .sort_values(by='reply_count',ascending=False)[:10].plot(kind='bar', figsize=(15, 7.5))
    plt.title(f'User with the most reaction in #{channel}', size=25);
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()



In [30]:
import os, sys
import re
import json
import glob
import datetime
from collections import Counter

# Get the current working directory
current_dir = os.getcwd()

# Get the parent directory (one level up)
parent_dir = os.path.dirname(current_dir)

# Construct the path to the 'network-analysis' directory
network_analysis_path = os.path.join(parent_dir, 'network-analysis')

# Append the path to sys.path
sys.path.append(network_analysis_path)


import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from wordcloud import WordCloud

from src.loader import *
from src.utils import *
from src.config import *

loaded_data = SlackDataLoader("anonymized")

usage: ipykernel_launcher [-h] [--output OUTPUT] --path PATH
                          [--channel CHANNEL] [--userfile USERFILE]
ipykernel_launcher: error: the following arguments are required: --path


SystemExit: 2

d:\10academy\week 0\network-analysis\venv3\lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Insight Extraction

Below are some useful questions to answer. Feel free to explore to answer other interesting questions that may be of help to get insight about student's behaviour, need, and future performance 

In [ ]:
# which user has the highest number of reply counts?

In [ ]:
# Visualize reply counts per user per channel

In [ ]:
# what is the time range of the day that most messages are sent?


In [ ]:
# what kind of messages are replied faster than others?

In [ ]:
# Relationship between # of messages and # of reactions

In [ ]:
# Classify messages into different categories such as questions, answers, comments, etc.

In [ ]:
# Which users got the most reactions?

In [ ]:
# Model topics mentioned in the channel

In [ ]:
# What are the topics that got the most reactions?

### Harder questions to look into

In [ ]:
# Based on messages, reactions, references shared, and other relevant data such as classification of questions into techical question, comment, answer, aorder stu the python, statistics, and sql skill level of a user?